In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import os
import cv2
from PIL import Image
import copy
import time
import pickle

import torch
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, TensorDataset, Subset, WeightedRandomSampler
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from tqdm import tqdm
import glob

In [3]:
loaded_model = torch.load('../input/inception-dr/inception_weights.h5', map_location=torch.device('cpu'))
loaded_model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [4]:
class ben_color(object):
    def __call__(self, img, sigmaX=10):
        """
        :param img: PIL): Image 

        :return: Normalized image
        """

        img = np.asarray(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.crop_image_from_gray(img)
        img = cv2.resize(img, (input_size, input_size))
        img = cv2.addWeighted (img, 4, cv2.GaussianBlur(img, (0,0), sigmaX), -4, 128)
        return Image.fromarray(img)

    def crop_image_from_gray(self, img, tol=7):
        if img.ndim ==2:
            mask = img>tol
            return img[np.ix_(mask.any(1),mask.any(0))]
        elif img.ndim==3:
            gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            mask = gray_img>tol
            
            check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
            if (check_shape == 0):
                return img 
            else:
                img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
                img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
                img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
                img = np.stack([img1,img2,img3],axis=-1)
            return img

    def __repr__(self):
        return self.__class__.__name__+'()'

In [5]:
def predict_image(image, test_transforms):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    _input = Variable(image_tensor)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    _input = _input.to(device)
    output = loaded_model(_input)
    x = output.data.cpu().numpy()[0]
    x = [float(i) for i in x]
    norm = [(i-min(x))/(max(x)-min(x)) for i in x]
    norm = [round(100*i/sum(norm), 2) for i in norm]
    pred = x.index(max(x))
    return norm



def transform_image(image_url):

    test_transforms = transforms.Compose([
        ben_color(),
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    image = Image.open(image_url)
    return predict_image(image, test_transforms)

In [7]:
df_0 = pd.DataFrame(columns=['path', 'real'])
df_0['path'] = glob.glob("../input/drunstratified/content/non_stratified_dr/0/*.*")
df_0['real'] = 0

df_1 = pd.DataFrame(columns=['path', 'real'])
df_1['path'] = glob.glob("../input/drunstratified/content/non_stratified_dr/1/*.*")
df_1['real'] = 1

df_2 = pd.DataFrame(columns=['path', 'real'])
df_2['path'] = glob.glob("../input/drunstratified/content/non_stratified_dr/2/*.*")
df_2['real'] = 2

df_3 = pd.DataFrame(columns=['path', 'real'])
df_3['path'] = glob.glob("../input/drunstratified/content/non_stratified_dr/3/*.*")
df_3['real'] = 3

df_4 = pd.DataFrame(columns=['path', 'real'])
df_4['path'] = glob.glob("../input/drunstratified/content/non_stratified_dr/4/*.*")
df_4['real'] = 4

In [8]:
df = pd.concat([df_0, df_1, df_2, df_3, df_4])

In [9]:
df

,path,real
0,../input/drunstratified/content/non_stratified...,0
1,../input/drunstratified/content/non_stratified...,0
2,../input/drunstratified/content/non_stratified...,0
3,../input/drunstratified/content/non_stratified...,0
4,../input/drunstratified/content/non_stratified...,0
...,...,...
1292,../input/drunstratified/content/non_stratified...,4
1293,../input/drunstratified/content/non_stratified...,4
1294,../input/drunstratified/content/non_stratified...,4
1295,../input/drunstratified/content/non_stratified...,4


In [11]:
input_size = 299
tqdm.pandas()
df.head(10).path.progress_apply(transform_image)

100%|██████████| 10/10 [00:03<00:00,  3.31it/s]


0    [37.06, 38.11, 21.89, 2.94, 0.0]
1     [43.6, 27.91, 22.77, 5.72, 0.0]
2     [46.3, 29.55, 22.92, 1.23, 0.0]
3    [50.46, 26.26, 20.61, 2.67, 0.0]
4    [42.1, 30.81, 16.89, 0.0, 10.19]
5    [39.38, 24.52, 29.13, 6.98, 0.0]
6     [51.04, 26.2, 19.11, 0.0, 3.65]
7     [46.4, 26.96, 22.02, 4.63, 0.0]
8    [46.97, 25.23, 27.08, 0.72, 0.0]
9    [43.67, 33.34, 22.64, 0.36, 0.0]
Name: path, dtype: object

In [12]:
df['preds'] = df.path.progress_apply(transform_image)

100%|██████████| 68069/68069 [5:15:12<00:00,  3.60it/s]  


In [28]:
df.to_csv('inference.csv', index=False)

In [25]:
df.reset_index(inplace=True)

In [26]:
split = pd.DataFrame(df["preds"].to_list(), columns=[i for i in range(5)])

In [27]:
df.join(split)

,level_0,index,path,real,preds,0,1,2,3,4
0,0,0,../input/drunstratified/content/non_stratified...,0,"[37.06, 38.11, 21.89, 2.94, 0.0]",37.06,38.11,21.89,2.94,0.00
1,1,1,../input/drunstratified/content/non_stratified...,0,"[43.6, 27.91, 22.77, 5.72, 0.0]",43.60,27.91,22.77,5.72,0.00
2,2,2,../input/drunstratified/content/non_stratified...,0,"[46.3, 29.55, 22.92, 1.23, 0.0]",46.30,29.55,22.92,1.23,0.00
3,3,3,../input/drunstratified/content/non_stratified...,0,"[50.46, 26.26, 20.61, 2.67, 0.0]",50.46,26.26,20.61,2.67,0.00
4,4,4,../input/drunstratified/content/non_stratified...,0,"[42.1, 30.81, 16.89, 0.0, 10.19]",42.10,30.81,16.89,0.00,10.19
...,...,...,...,...,...,...,...,...,...,...
68064,68064,1292,../input/drunstratified/content/non_stratified...,4,"[8.11, 0.0, 34.34, 20.71, 36.83]",8.11,0.00,34.34,20.71,36.83
68065,68065,1293,../input/drunstratified/content/non_stratified...,4,"[0.0, 5.71, 41.79, 26.84, 25.66]",0.00,5.71,41.79,26.84,25.66
68066,68066,1294,../input/drunstratified/content/non_stratified...,4,"[9.3, 0.0, 49.98, 27.63, 13.1]",9.30,0.00,49.98,27.63,13.10
68067,68067,1295,../input/drunstratified/content/non_stratified...,4,"[0.0, 5.98, 24.43, 17.16, 52.42]",0.00,5.98,24.43,17.16,52.42


In [36]:
df.drop('index', axis=1).join(split).drop('preds', axis=1).to_csv("final_inference.csv",index=False)